# Language modelling


Обучим две различные символьные модели для генерации динозавров:
* модель на символьных биграмах
* ***RNN***-модель.


## Bigram model


In [1]:
!wget https://raw.githubusercontent.com/artemovae/NLP-seminar-LM/master/dinos.txt

--2020-02-27 00:24:00--  https://raw.githubusercontent.com/artemovae/NLP-seminar-LM/master/dinos.txt
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 151.101.84.133
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|151.101.84.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 19909 (19K) [text/plain]
Сохранение в: «dinos.txt»

dinos.txt           100%[===================>]  19,44K  --.-KB/s    за 0,05s   

2020-02-27 00:24:00 (408 KB/s) - «dinos.txt» сохранён [19909/19909]



In [2]:
!cat dinos.txt | tail

Zhuchengtyrannus
Ziapelta
Zigongosaurus
Zizhongosaurus
Zuniceratops
Zunityrannus
Zuolong
Zuoyunlong
Zupaysaurus
Zuul

In [3]:
names = ['<' + name.strip().lower() + '>' for name in open('dinos.txt').readlines()]
print(names[:10])

['<aachenosaurus>', '<aardonyx>', '<abdallahsaurus>', '<abelisaurus>', '<abrictosaurus>', '<abrosaurus>', '<abydosaurus>', '<acanthopholis>', '<achelousaurus>', '<acheroraptor>']


In [4]:
import nltk

Вычислим частоту каждого символа в корпусе имен динозавров

In [5]:
chars = [char for name in names for char in name]
freq = nltk.FreqDist(chars)

In [6]:
freq

FreqDist({'a': 2487, 's': 2285, 'u': 2123, 'o': 1710, 'r': 1704, '<': 1536, '>': 1536, 'n': 1081, 'i': 944, 'e': 913, ...})

In [7]:
print(list(freq.keys()))

['<', 'a', 'c', 'h', 'e', 'n', 'o', 's', 'u', 'r', '>', 'd', 'y', 'x', 'b', 'l', 'i', 't', 'p', 'v', 'm', 'g', 'f', 'j', 'k', 'w', 'z', 'q']


In [8]:
freq.most_common(10)

[('a', 2487),
 ('s', 2285),
 ('u', 2123),
 ('o', 1710),
 ('r', 1704),
 ('<', 1536),
 ('>', 1536),
 ('n', 1081),
 ('i', 944),
 ('e', 913)]

Define a function to estimate probabilty of character

In [9]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l

In [10]:
print('p(a) = %1.4f' %unigram_prob('a'))

p(a) = 0.1160


Вычислим условную вероятность каждого символа в зависимости от того, какой символ стоял на предыдущей позиции.

In [11]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))

In [12]:
cfreq['a']

FreqDist({'u': 791, 'n': 347, 't': 204, 's': 171, 'l': 138, '>': 138, 'r': 124, 'c': 100, 'p': 89, 'm': 68, ...})

Оценим условные вероятности с помощью MLE.

In [13]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)

In [14]:
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

p(a a) = 0.0044
p(a b) = 0.0097
p(a u) = 0.3181


In [15]:
cprob['a'].generate()

'h'

### Task 1.
a. Write a function to generate a dinosaur name of **fixed** length. Use '<' as a start of name symbol.

b. Write a function to generate a dinosaur names of any length. 

In [16]:
def generate_n_word(n=10):
    word = '<'
    for i in range(n):
        word += cprob[word[-1]].generate()
    word += '>'
    return word

In [17]:
generate_n_word()

'<plitopnere>'

## Реккурентные нейронные сети (RNN)

Исходная последовательность:

$x_{1:n} = x_1, x_2, \ldots, x_n$, $x_i \in \mathbb{R}^{d_{in}}$

Для каждого входного значения $x_{1:i}$ получаем на выходе $y_i$:

$y_i = RNN(x_{1:i})$, $y_i \in \mathbb{R}^{d_{out}}$

Для всей последовательности $x_{1:n}$:

$y_{1:n} = RNN^{*}(x_{1:n})$, $y_i \in \mathbb{R}^{d_{out}}$

$R$ - рекурсивная функция активации, зависящая от двух параметров: $x_i$ и $s_{i-1}$ (вектор предыдущего состояния)

$RNN^{*}(x_{1:n}, s_0) = y_{1:n}$

$y_i = O(s_i) = g(W^{out}[s_{i} ,x_i] +b)$

$s_i = R(s_{i-1}, x_i)$

$s_i = R(s_{i-1}, x_i) = g(W^{hid}[s_{i-1} ,x_i] +b)$  -- конкатенация $[s_{i-1}, x]$

$x_i \in \mathbb{R}^{d_{in}}$, $y_i \in \mathbb{R}^{ d_{out}}$, $s_i \in \mathbb{R}^{d_{hid}}$

$W^{hid} \in \mathbb{R}^{(d_{in}+d_{out}) \times d_{hid}}$, $W^{out} \in \mathbb{R}^{d_{hid} \times d_{out}}$

Построим языковую модель на основе RNN с помощью pytorch

In [18]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

torch.set_printoptions(linewidth=200)

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 50

Подготовим данные

In [20]:
class DinosDataset(Dataset):
    def __init__(self):
        super().__init__()
        with open('dinos.txt') as f:
            content = f.read().lower()
            self.vocab = sorted(set(content)) + ['<', '>']
            self.vocab_size = len(self.vocab)
            self.lines = content.splitlines()
        self.ch_to_idx = {c:i for i, c in enumerate(self.vocab)}
        self.idx_to_ch = {i:c for i, c in enumerate(self.vocab)}
    
    def __getitem__(self, index):
        line = self.lines[index]
        #teacher forcing
        x_str = '<' + line 
        y_str = line + '>' 
        x = torch.zeros([len(x_str), self.vocab_size], dtype=torch.float)
        y = torch.empty(len(x_str), dtype=torch.long)
        for i, (x_ch, y_ch) in enumerate(zip(x_str, y_str)):
            x[i][self.ch_to_idx[x_ch]] = 1
            y[i] = self.ch_to_idx[y_ch]
        
        return x, y
    
    def __len__(self):
        return len(self.lines)

In [21]:
trn_ds = DinosDataset()
trn_dl = DataLoader(trn_ds, shuffle=True)

In [22]:
trn_ds.lines[1]

'aardonyx'

In [23]:
print(trn_ds.idx_to_ch)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: '<', 28: '>'}


In [24]:
trn_ds.vocab_size

29

In [25]:
x, y = trn_ds[1]

In [26]:
x.shape

torch.Size([9, 29])

In [27]:
y.shape

torch.Size([9])

In [28]:
y

tensor([ 1,  1, 18,  4, 15, 14, 25, 24, 28])

Опишем модель, функцию потерь и алгоритм оптимизации

In [29]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.3)
        self.i2o = nn.Linear(hidden_size, output_size)
    
    def forward(self, h_prev, x):
        combined = torch.cat([h_prev, x], dim = 1) # concatenate x and h
        h = torch.tanh(self.dropout(self.i2h(combined)))
        y = self.i2o(h)
        return h, y

In [30]:
model = RNN(trn_ds.vocab_size, hidden_size, trn_ds.vocab_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)

![rnn](images/dinos3.png)

In [31]:
def sample(model):
    model.eval()
    word_size=0
    newline_idx = trn_ds.ch_to_idx['>']
    with torch.no_grad():
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x = h_prev.new_zeros([1, trn_ds.vocab_size])
        start_char_idx = trn_ds.ch_to_idx['<']
        indices = [start_char_idx]
        x[0, start_char_idx] = 1
        predicted_char_idx = start_char_idx
        
        while predicted_char_idx != newline_idx and word_size != 50:
            h_prev, y_pred = model(h_prev, x)
            y_softmax_scores = torch.softmax(y_pred, dim=1)
            
            np.random.seed(np.random.randint(1, 5000))
            idx = np.random.choice(np.arange(trn_ds.vocab_size), p=y_softmax_scores.cpu().numpy().ravel())
            indices.append(idx)
            
            x = (y_pred == y_pred.max(1)[0]).float()
            
            predicted_char_idx = idx
            
            word_size += 1
        
        if word_size == 50:
            indices.append(newline_idx)
    return indices

In [32]:
def print_sample(sample_idxs):
    [print(trn_ds.idx_to_ch[x], end ='') for x in sample_idxs]
    print()

Обучим получившуюся модель

In [33]:
def train_one_epoch(model, loss_fn, optimizer):
    model.train()
    for line_num, (x, y) in enumerate(trn_dl):
        loss = 0
        optimizer.zero_grad()
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x, y = x.to(device), y.to(device)
        for i in range(x.shape[1]):
            h_prev, y_pred = model(h_prev, x[:, i])
            loss += loss_fn(y_pred, y[:, i])
            
        if (line_num+1) % 100 == 0:
            print_sample(sample(model))
        loss.backward()
        optimizer.step()

In [34]:
def train(model, loss_fn, optimizer, dataset='dinos', epochs=1):
    for e in range(1, epochs+1):
        print('Epoch:{}'.format(e))
        train_one_epoch(model, loss_fn, optimizer)
        print()

In [35]:
train(model, loss_fn, optimizer, epochs = 10)

Epoch:1
<oyrouiriororrhnsanrhsc>
<znsvpcuooa>
<anxqtastteusus>
<tnpstainalrcrluaaunanhtnibtclyouautunkrtl>
<tcurue>
<lamraseueos>
<annureasds>
<ptjrus>
<dsonysausuh>
<lbmuatusas>
<laidoinasaurus>
<tuson>
<snxusuuras>
<alrasaugis>
<aeirinaugus>

Epoch:2
<ueropmesausus>
<sraroaaurus>
<aaurasdurus>
<antotaurus>
<qplysdurui>
<lcgrasisaoruh>
<colicohrus>
<bttsinaur>
<snnstauros>
<ailtaasairus>
<gauaitaurus>
<llrsivopostauris>
<atetagaurus>
<gcbsanuurus>
<sncsnttus>

Epoch:3
<suanhantatn>
<aeolapatopc>
<anurrlaurus>
<sawrnstodaurus>
<sltagooaaurus>
<tcivosaurus>
<rsixtrauros>
<alrbthsagrup>
<csinaurus>
<ugttsaurus>
<sessuaeurus>
<seuagsairus>
<gcsao>
<nuatttaurus>
<snottbgsaurus>

Epoch:4
<tairua>
<dtnrasaurus>
<suroaurus>
<lroubaurus>
<slradgumus>
<gbctanturus>
<snesousus>
<suaioaurus>
<uacoobaurus>
<santotaurus>
<rrgysaurus>
<kcercauras>
<laihtnarrus>
<puatrsaurus>
<senpuasaurus>

Epoch:5
<euaasiangtaurus>
<wssasaurus>
<jypnosaorus>
<scrktap>
<laigopamris>
<puesusaurus>
<siottauras>
<amlsa

In [36]:
def generate_dino(n=50):
    word = '<'
    for i in range(n):
        word += cprob[word[-1]].generate()
        if word[-1] == '>':
            return word
    word += '>'
    return word

In [37]:
nn.RNN

torch.nn.modules.rnn.RNN

In [39]:
for i in range(10):
    print(generate_dino())

<juss>
<s>
<pteocerururhernthatarhicenerusurudenatechuusambidn>
<benasurthisiaus>
<ptracusanis>
<phodzaceinipurusakelocerausurus>
<da>
<mopakedos>
<gsas>
<tememedelites>
